In [ ]:
!nvidia-smi # reset until p100

Thu Apr 28 05:53:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
my_cluster = 0

clusters = [1, 1, 2, 1, 0, 2, 1, 1, 0, 1, 1, 2, 0, 1, 1, 2, 0, 1, 2, 1, 1, 2, 1, 0, 2, 2]
species = ['Barbary_macaque', 'Bonobo', 'Chacma_baboon', 'Chimpanzee',
       'Common_marmoset', 'Cotton-top_tamarin', 'Crab-eating_macaque',
       'Dusky_leaf_monkey', 'Emperor_tamarin', 'Formosan_rock_macaque',
       'Gibbon', 'Golden_lion_tamarin', 'Golden_snub-nosed_monkey',
       'Gorilla', 'Hamadryas_baboon', 'Japanese_macaque',
       'Lion-tailed_macaque', 'Mandrill', 'Olive_baboon', 'Orangutan',
       'Proboscis_monkey', 'Rhesus_macaque', 'Siamang', 'Squirrel_monkey',
       'Tufted_capuchin', 'Vervet_monkey']

my_species = []
for i in range(len(clusters)):
  if clusters[i] == my_cluster:
    my_species.append(species[i])
my_species

['Common_marmoset',
 'Emperor_tamarin',
 'Golden_snub-nosed_monkey',
 'Lion-tailed_macaque',
 'Squirrel_monkey']

In [ ]:
%%capture
! pip install deeplabcut==2.2.0.6
import deeplabcut

In [ ]:
! cp -r /content/drive/MyDrive/5561/baseline ./baseline

In [ ]:
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Zips/train_cropped.zip train_cropped.zip
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Zips/val_cropped.zip val_cropped.zip
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Zips/test_cropped.zip test_cropped.zip

In [ ]:
%%capture
! unzip train_cropped.zip
! unzip val_cropped.zip
! unzip test_cropped.zip

In [ ]:
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Annotations/train_annotation.json train_annotation.json
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Annotations/val_annotation.json val_annotation.json
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Annotations/test_prediction.json test_prediction.json

In [ ]:
import json

with open("train_annotation.json") as f:
  train_js = json.load(f)

with open("val_annotation.json") as f:
  val_js = json.load(f)

with open("test_prediction.json") as f:
  test_js = json.load(f)

In [ ]:
import numpy as np
species = list(np.unique([example['species'] for example in train_js['data']]))

In [ ]:
def get_cropped_name(name):
  return name[:-11] + "cropped_" + name[-11:]

get_cropped_name('train_0000000.jpg')

'train_cropped_0000000.jpg'

In [ ]:
def get_img_names(species):
  names = []
  for train in train_js['data']:
    if train['species'] in species:
      names.append(get_cropped_name(train['file']))
  for val in val_js['data']:
    if val['species'] in species:
      names.append(get_cropped_name(val['file']))
  return names

len(get_img_names(["Gorilla"]))

1778

In [ ]:
def get_img_names_test(species):
  names = []
  for test in test_js['data']:
    if test['species'] in species:
      names.append(get_cropped_name(test['file']))
  return names

get_img_names_test(["Gorilla"])[:10]

['test_cropped_0000112.jpg',
 'test_cropped_0000142.jpg',
 'test_cropped_0000245.jpg',
 'test_cropped_0000277.jpg',
 'test_cropped_0000298.jpg',
 'test_cropped_0000328.jpg',
 'test_cropped_0000338.jpg',
 'test_cropped_0000356.jpg',
 'test_cropped_0000396.jpg',
 'test_cropped_0000422.jpg']

In [ ]:
import os
import shutil

# for s in species:
shutil.copytree("baseline", f"baseline_{my_cluster}")

'baseline_0'

In [ ]:
# for s in species:
#   os.mkdir(f"test_cropped_{s}")
os.mkdir(f"test_cropped_{my_cluster}")

In [ ]:
def crop_df(df, imgs):
  return df[df.index.get_level_values(2).isin(imgs)]

In [ ]:
import pandas as pd

# for s in species:
#   dir = f"baseline_{s}/"
#   csv = dir + "labeled-data/baseline/CollectedData_daniel.csv"
#   h5 = dir + "labeled-data/baseline/CollectedData_daniel.h5"
#   df = pd.read_hdf(h5)
#   imgs = species_to_imgs[s]
#   df_cropped = crop_df(df, imgs)
#   df_cropped.to_csv(csv)
#   df_cropped.to_hdf(h5, mode="w", key="label")



dir = f"baseline_{my_cluster}/"
csv = dir + "labeled-data/baseline/CollectedData_daniel.csv"
h5 = dir + "labeled-data/baseline/CollectedData_daniel.h5"
df = pd.read_hdf(h5)
imgs = get_img_names(my_species)
df_cropped = crop_df(df, imgs)
df_cropped.to_csv(csv)
df_cropped.to_hdf(h5, mode="w", key="label")

In [ ]:
# Move images

import os
import shutil

imgs = get_img_names(my_species)
for img in imgs:
  prefix = "train_cropped/" if "train" in img else "val_cropped/"
  shutil.move(prefix + img, f"baseline_{my_cluster}/labeled-data/baseline/{img}")

In [ ]:
imgs = get_img_names_test(my_species)
for img in imgs:
  shutil.move(f"test_cropped/{img}", f"test_cropped_{my_cluster}/{img}")

In [ ]:
path_config_file = f"/content/baseline_{my_cluster}/config.yaml"
print(f"training model for cluster {my_cluster}")

deeplabcut.create_training_dataset(path_config_file, augmenter_type="imgaug")

deeplabcut.train_network(path_config_file, max_snapshots_to_keep=5, displayiters=100, saveiters=10000, maxiters=350000)

deeplabcut.analyze_videos(path_config_file,
                          [f'/content/test_cropped_{my_cluster}'], 
                          videotype='.jpg', 
                          destfolder=f"/content/test_predictions_{my_cluster}")


In [ ]:
shutil.make_archive(f"/content/drive/MyDrive/5561/3-models/test_predictions_{my_cluster}", 'zip', f"/content/test_predictions_{my_cluster}")

'/content/drive/MyDrive/5561/3-models/test_predictions_0.zip'

In [ ]:
while True:
  pass

KeyboardInterrupt: ignored